## Variational auto encoders
Variational auto encoders have a slightly different approach than traditional auto encoders. First, instead of outputing the latent vector, the encoder outputs the parameters of a probablity distribution from which the latent vector is drawn.

This has a few advantages: first, it makes the output vectors easily interpretable and you can generate more similar images from the same input image - remember that there is some inherent randomness to drawing from a probability distribution. The same input image will produce the same output parameters, but they are just the parameters of distributions and the decoder can cope with variations that are consistent with the distribution.

Second, because we add a training term to drive the distribution parameters towards gaussians, we end up with a compact, local latent space. Each digit's latent space is close to the other digits, making interpolation between samples easy.

Third, because the sample space is generally kept compact, and the input to the decoder is of a size consistent with the probability distrubtions, we can generate new samples much more easily without the encoder.

In [3]:
# Load numpy and tensorflow:

import tensorflow as tf
import numpy
import random
tf.enable_eager_execution()

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import display, clear_output






In [8]:
class ResBlock(tf.keras.models.Model):
    
    '''
    Residual block with configurable activation
    '''
    
    def __init__(self, n_filters, activation=tf.nn.relu):
        tf.keras.models.Model.__init__(self)

        self.convolution1 = tf.keras.layers.Convolution2D(
            kernel_size = [3, 3], 
            filters     = n_filters,
            padding     = "same",
            activation  = None,
        )
        
        self.convolution2 = tf.keras.layers.Convolution2D(
            kernel_size = [3, 3], 
            filters     = n_filters,
            padding     = "same",
            activation  = None,
        )
        
        self.batch_norm1 = tf.keras.layers.BatchNormalization()
        self.batch_norm2 = tf.keras.layers.BatchNormalization()


        self.activation = activation
        
    def call(self, inputs):
        
        # Conv 1
        x = self.convolution1(inputs)
        x = self.batch_norm1(x)
        x = self.activation(x)
        
        # Conv2
        x = self.convolution2(x)
        
        x = self.batch_norm2(x)
        
        # Sum:
        x = x + inputs 
        
        x = self.activation(x)
        
        return x
    
    
class ConvolutionalEncoder(tf.keras.models.Model):
    '''
    Simple autoencoder forward layer
    '''
    
    def __init__(self, activation=tf.nn.relu, latent_size = 10):
        tf.keras.models.Model.__init__(self)
        self._latent_size = latent_size

        # Apply a 5x5 kernel to the image:
        self.encoder_layer_1 = ResBlock(activation=activation, n_filters=64)
        
        # Use a 2x2 kernel of stride 2x2 to downsample:
        self.pool_1 = tf.keras.layers.MaxPool2D()
        
        self.encoder_layer_2 = ResBlock(activation=activation, n_filters=64)
        self.pool_2 = tf.keras.layers.MaxPool2D()
        
        self.encoder_layer_3 = ResBlock(activation=activation, n_filters=64)
        self.pool_3 = tf.keras.layers.MaxPool2D()

        self.encoder_layer_final = tf.keras.layers.Dense(
            units = latent_size,
            activation = None,
            )
        
        
        


    def call(self, inputs):
        
        batch_size = inputs.shape[0]
        x = inputs
        # Make sure the input is the right shape:
        x = tf.reshape(x, [batch_size, 28, 28, 1])
            
        x = self.encoder_layer_1(x)
        x = self.pool_1(x)
        x = self.encoder_layer_2(x)
        x = self.pool_2(x)
        x = self.encoder_layer_3(x)
        x = self.pool_3(x)
        x = tf.keras.layers.Flatten()(x)
        x = self.encoder_layer_final(x)
        return tf.reshape(x, [batch_size, self._latent_size])



class VariationalEncoder(tf.keras.models.Model):
    
    def __init__(self, activation = tf.nn.relu, latent_size = 10):
        tf.keras.models.Model.__init__(self)
        
        self.latent_size = latent_size
        self.base_encoder = ConvolutionalEncoder(activation, latent_size*2)

    def call(self, inputs):
        
        # First use the main auto encoder:
        x = self.base_encoder(inputs)
        # Then, reshape into the right size:
        x = tf.reshape(x, [x.shape[0], self.latent_size, 2])
        
        # And split the means and widths:
        means  = x[:,:,0]
        widths = x[:,:,1]
        
        return means, widths
        # Create normal distributions:

        
class ConvolutionalDecoder(tf.keras.models.Model):
    
    def __init__(self, activation=tf.nn.tanh):
        tf.keras.models.Model.__init__(self)

        # The decoder runs the encoder steps but in reverse.
        
        # The first step is a dense layer to get to the right number of units.
        # It turns out to need to be of shape [3, 3]
        self.dense = tf.keras.layers.Dense(
            units = 3 * 3 * 64
        )
        
        
        self.decoder_layer_1 = ResBlock(n_filters=64, activation=activation)
        
        self.unpool_1 = tf.keras.layers.UpSampling2D(
            size          = 2,
            interpolation = "nearest",
        )

        self.decoder_layer_2 = ResBlock(n_filters=64, activation=activation)
        
        self.unpool_2 = tf.keras.layers.UpSampling2D(
            size          = 2,
            interpolation = "nearest",
        )


        self.decoder_layer_3 = ResBlock(n_filters=64, activation=activation)
        
        self.decoder_layer_final = tf.keras.layers.Convolution2D(
            kernel_size = [1, 1],
            filters     = 1,
            padding     = "same",
            use_bias    = True,
            kernel_regularizer = tf.contrib.layers.l2_regularizer,
            activation  = None,
        )
        

        
    def call(self, inputs):
        ''' 
        Reshape at input and output: 
        '''
            
        
        batch_size = inputs.shape[0]


        x = self.dense(inputs)


        # First Step is to to un-pool the encoded state into the right shape:
        x = tf.reshape(x, [batch_size, 3, 3, 64])
        x = tf.image.resize_images(
            x,
            size=[7,7]
        )
        
        x = self.decoder_layer_1(x)
        x = self.unpool_1(x)
        x = self.decoder_layer_2(x)
        x = self.unpool_2(x)
        x = self.decoder_layer_3(x)
        x = self.decoder_layer_final(x)

        x = tf.nn.sigmoid(x)
        
        x = tf.reshape(x, [batch_size, 28, 28, 1])
        return x

In [12]:
vae =VariationalEncoder()
vad = ConvolutionalDecoder()

In [13]:
# Reloading the dataset here to make sure it's shuffled properly
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(numpy.float32) * (1./256)
x_test  = x_test.astype(numpy.float32) * (1./256)
y_train = y_train.astype(numpy.int64)
y_test  = y_test.astype(numpy.int64)

In [14]:
example = x_train[0].reshape(1,28,28,1)
means, widths = vae(example)

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
print(means)
print(widths)

tf.Tensor(
[[ 1.0482228  -0.406279   -2.6046536  -3.3866324   0.91251147  3.492838
  -2.3752315   1.7152349  -1.8961774  -0.07210225]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[-1.004668   1.363038   0.5620045 -0.9579761  1.1902835 -0.4230429
  -1.6915435  1.5790937 -0.3119232  0.8879189]], shape=(1, 10), dtype=float32)


In [19]:
N_EPOCHS=1
BATCH_SIZE=64

In [21]:
optimizer = tf.train.AdamOptimizer()

# We will keep track of loss along the way:
steps = []
loss_history = []
test_loss_history = []
test_steps = []

# Define a figure here which we can update along the way:
fig       = plt.figure(figsize=(16,9))
# Create 2x2 sub plots
gs = gridspec.GridSpec(2, 2)
ax_loss   = plt.subplot(gs[:,0])
ax_input  = plt.subplot(gs[0,1])
ax_output = plt.subplot(gs[1,1])


for e in range(N_EPOCHS):
    
    # First, shuffle the training data:
    numpy.random.shuffle(x_train)
    
    # Now, loop over the data in batches:
    for batch in range(int(60000/BATCH_SIZE)):
    
        # Variable for plotting:
        epoch_value = e + (batch*BATCH_SIZE)/60000
    
        # Load some data:
        batch_images = x_train[batch*BATCH_SIZE : (batch+1)*BATCH_SIZE]
        # Reshape the data:
        batch_images = batch_images.reshape([BATCH_SIZE, 28,28,1])
        with tf.GradientTape() as tape:
            means, widths = vae(batch_images)
            intermediate_state = tf.random.normal(shape=means.shape)
            intermediate_state = intermediate_state * tf.exp(widths * .5) + means
            
            decoded_images = vad(intermediate_state)
            
            
            loss_encoder = tf.losses.mean_squared_error(batch_images, decoded_images)
            loss_dists   = tf.reduce_sum(means*means + widths*widths)
            loss_dists  -= tf.reduce_sum(tf.log(widths)) + 1
            loss_value = loss_encoder + loss_dists
        
        # Collect the trainable variables as the union of the encoder and decoder variables:
        trainable_vars = vae.trainable_variables + vad.trainable_variables

        # Update the loss history:
        loss_history.append(loss_value.numpy())
        steps.append(epoch_value)

        # Apply the update to the model:
        grads = tape.gradient(loss_value, trainable_vars)
        optimizer.apply_gradients(zip(grads, trainable_vars),
                                 global_step=tf.train.get_or_create_global_step())


            
        if batch % 3 == 0:
            test_indexes = numpy.random.choice(range(10000), size=BATCH_SIZE)
            test_images = x_test[test_indexes]
            test_images = test_images.reshape([BATCH_SIZE, 28,28,1])
            
            test_means, test_widths = vae(test_images)
            test_intermediate_state = tf.random.normal(shape=test_means.shape)
            test_intermediate_state = test_intermediate_state * tf.exp(test_widths * .5) + test_means
            
            test_decoded_images = vad(test_intermediate_state)
            
            
            test_loss_encoder = tf.losses.mean_squared_error(test_images, test_decoded_images)
            test_loss_dists   = tf.reduce_sum(test_means*test_means + test_widths*test_widths)
            test_loss_dists  -= tf.reduce_sum(tf.log(test_widths)) + 1
            test_loss_value = test_loss_encoder + test_loss_dists
            
            test_loss_history.append(test_loss_value.numpy())
            test_steps.append(epoch_value)

            # Here we update the plots.  Replot the loss, and show the latest input and output images
            ax_loss.set_xlim(0, 1.1*epoch_value)
            ax_loss.cla()
            ax_loss.plot(steps, loss_history, label="Train Loss")
            ax_loss.plot(test_steps, test_loss_history, label="Test Loss")
            ax_loss.grid(True)
            ax_loss.legend(fontsize=25)

            # Display an input image:
            ax_input.imshow(batch_images[0].reshape(28, 28),vmin=0, vmax=1)

            # Display an output image:
            ax_output.imshow(decoded_images.numpy()[0].reshape(28, 28),vmin=0, vmax=1)

            display(fig)
            clear_output(wait = True)
            plt.pause(0.25)

        
# Let's save the models weights as well:
encoder.save_weights("vae_encoder_basic.h5")
decoder.save_weights("vae_decoder_basic.h5")




KeyboardInterrupt: 